## Sudoku AI

The code is written in the cell below.

Testing can be done automatically in the cell after.

In [6]:
import numpy as np
import re

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    
    columnConstraints = [[i for i in range(1, 10)] for _ in range(0, 9)]
    rowConstraints = [[i for i in range(1, 10)] for _ in range(0, 9)]
    squareConstraints = [[i for i in range(1, 10)] for _ in range(0, 9)]
    empty = []
    individualConstraints = dict()
    stack = []
    
    for i in range (0, len(sudoku)):
        for j in range(0, len(sudoku)):
            if(sudoku[i][j] == 0):
                empty.append([i,j])
            else:
                if(sudoku[i][j] in rowConstraints[i]):
                    rowConstraints[i].remove(sudoku[i][j])
                elif (sudoku[i][j] not in rowConstraints[i]):
                    return [[-1 for i in range(1,10)] for _ in range(0,9)]
                if(sudoku[i][j] in columnConstraints[j]):
                    columnConstraints[j].remove(sudoku[i][j])
                elif (sudoku[i][j] not in columnConstraints[j]):
                    return [[-1 for i in range(1,10)] for _ in range(0,9)]
                if(sudoku[i][j] in squareConstraints[j//3 + (i//3 * 3)]):
                    squareConstraints[j//3 + (i//3 * 3)].remove(sudoku[i][j])
                elif (sudoku[i][j] not in squareConstraints[j//3 + (i//3 * 3)]):
                    return [[-1 for i in range(1,10)] for _ in range(0,9)]
                    
    for each in empty:
        individualConstraints[''.join(str(x) for x in each)] = set.intersection(set(columnConstraints[each[1]]), set(rowConstraints[each[0]]), set(squareConstraints[each[1]//3 + (each[0]//3 * 3)]))    
    
    solution = depthFirst(sudoku, getNode(empty, individualConstraints), empty, individualConstraints, rowConstraints, columnConstraints, squareConstraints, stack)
    if np.all(solution == False):
        return [[-1 for i in range(1,10)] for _ in range(0,9)]
    return solution
    
    
def depthFirst(sudoku, unitSquare, empty, individualConstraints, rowConstraints, columnConstraints, squareConstraints, stack):
    if unitSquare == None and finalInvalid(sudoku) == False:
        return sudoku
    elif unitSquare == None:
        return False 
    test = individualConstraints[''.join(str(x) for x in unitSquare)]
    if len(test) == 1:
        if solved(empty):
            return sudoku
        else:
            val = next(iter(test))
            sudoku[unitSquare[0]][unitSquare[1]] = val            
            empty.remove(unitSquare)
            newConstraints = dict(updateConstraints(individualConstraints, unitSquare, val, rowConstraints, columnConstraints, squareConstraints, stack))
            nextNode = getNode(empty, individualConstraints)
            sol = depthFirst(sudoku, nextNode, empty, newConstraints, rowConstraints, columnConstraints, squareConstraints, stack)
            if np.all(sol!= False):
                return sol
            empty.append(unitSquare)
            sudoku[unitSquare[0]][unitSquare[1]] = '0'
            individualConstraints = restoreConstraints(individualConstraints, unitSquare, val, stack, rowConstraints, columnConstraints, squareConstraints)
            return False

    elif len(test)>1:
        for each in test:
            sudoku[unitSquare[0]][unitSquare[1]] = each
            empty.remove(unitSquare)
            newConstraints = dict(updateConstraints(individualConstraints, unitSquare, each, rowConstraints, columnConstraints, squareConstraints, stack))
            nextNode = getNode(empty, individualConstraints)
            sol = depthFirst(sudoku, nextNode, empty, newConstraints, rowConstraints, columnConstraints, squareConstraints, stack)
            if np.all(sol!= False):
                return sol
            else:
                empty.append(unitSquare)
                individualConstraints = restoreConstraints(individualConstraints, unitSquare, each, stack, rowConstraints, columnConstraints, squareConstraints)
                sudoku[unitSquare[0]][unitSquare[1]] = '0'
    
    else:
        return False
            
    return False
    
def getNode(empty, individualConstraints):
    minNo = 999
    if(len(empty) > 0):
        for each in empty:
            if len(individualConstraints[''.join(str(x) for x in each)]) < minNo:
                minNo = len(individualConstraints[''.join(str(x) for x in each)])
                minNode = each
        if minNo > 0:
            return minNode
        else:
            return None
    else:
        return None
    
def updateConstraints(individualConstraints, unitSquare, each, rowConstraints, columnConstraints, squareConstraints, stack):

    rowStack = []
    columnStack = []
    squareStack = []
    combinationStack = []
    
    square = unitSquare[1]//3 + (unitSquare[0]//3 * 3)
    if square == 0:
        iMax = 2
        jMax = 2
        iMin = 0
        jMin = 0
    elif square == 1:
        iMax = 2
        jMax = 5
        iMin = 0
        jMin = 3
    elif square == 2:
        iMax = 2
        jMax = 8
        iMin = 0
        jMin = 6
    elif square == 3:
        iMax = 5
        jMax = 2
        iMin = 3
        jMin = 0
    elif square == 4:
        iMax = 5
        jMax = 5
        iMin = 3
        jMin = 3
    elif square == 5:
        iMax = 5
        jMax = 8
        iMin = 3
        jMin = 6
    elif square == 6:
        iMax = 8
        jMax = 2
        iMin = 6
        jMin = 0
    elif square == 7:
        iMax = 8
        jMax = 5
        iMin = 6
        jMin = 3
    elif square == 8:
        iMax = 8
        jMax = 8
        iMin = 6
        jMin = 6
    
    for key in individualConstraints.keys():
        if re.match("[0-9]"+str(unitSquare[1]), key):
            individualConstraints.update({key:individualConstraints[key] - ({each}).intersection(individualConstraints[key])})
        elif re.match(str(unitSquare[0])+"[0-9]", key):
            individualConstraints.update({key:individualConstraints[key] - ({each}).intersection(individualConstraints[key])})
        elif re.match("[" + str(iMin) + "-" + str(iMax) + "]" + "[" + str(jMin) + "-" + str(jMax) + "]", key):
            individualConstraints.update({key:individualConstraints[key] - ({each}).intersection(individualConstraints[key])})
    
    for x in range(0, 9):
        if each in rowConstraints[x]:
            rowStack.append(1)
        else:
            rowStack.append(0)
            
    for y in range(0, 9):
        if each in columnConstraints[y]:
            columnStack.append(1)
        else:
            columnStack.append(0)
    
    for q in range(0,9):
        if each in squareConstraints[q]:
            squareStack.append(1)
        else:
            squareStack.append(0)
    
    for z in range(0,9):
        combinationStack.append([rowStack[z], columnStack[z], squareStack[z]])
    
    stack.append(combinationStack)
    if each in rowConstraints[unitSquare[0]]:
        rowConstraints[unitSquare[0]].remove(each)
    if each in columnConstraints[unitSquare[1]]:
        columnConstraints[unitSquare[1]].remove(each)
    squareConstraints[square].remove(each)
    
    return individualConstraints


def restoreConstraints(individualConstraints, unitSquare, each, stack, rowConstraints, columnConstraints, squareConstraints):
    square = unitSquare[1]//3 + (unitSquare[0]//3 * 3)
    test = stack.pop()
    if square == 0:
        iMax = 2
        jMax = 2
        iMin = 0
        jMin = 0
    elif square == 1:
        iMax = 2
        jMax = 5
        iMin = 0
        jMin = 3
    elif square == 2:
        iMax = 2
        jMax = 8
        iMin = 0
        jMin = 6
    elif square == 3:
        iMax = 5
        jMax = 2
        iMin = 3
        jMin = 0
    elif square == 4:
        iMax = 5
        jMax = 5
        iMin = 3
        jMin = 3
    elif square == 5:
        iMax = 5
        jMax = 8
        iMin = 3
        jMin = 6
    elif square == 6:
        iMax = 8
        jMax = 2
        iMin = 6
        jMin = 0
    elif square == 7:
        iMax = 8
        jMax = 5
        iMin = 6
        jMin = 3
    elif square == 8:
        iMax = 8
        jMax = 8
        iMin = 6
        jMin = 6
    
    
    for key in individualConstraints.keys():
        if re.match("[0-9]"+str(unitSquare[1]), key) and test[int(key[0])][0] == 1 and test[int(key[1])][1] == 1 and test[int(key[1])//3 + (int(key[0])//3 * 3)][2] == 1 and key != (str(unitSquare[0]) + str(unitSquare[1])):
            individualConstraints.update({key: ({each}).union(individualConstraints[key])})
        elif re.match(str(unitSquare[0])+"[0-9]", key) and test[int(key[0])][0] == 1 and test[int(key[1])][1] == 1 and test[int(key[1])//3 + (int(key[0])//3 * 3)][2] == 1 and  key != (str(unitSquare[0]) + str(unitSquare[1])):
            individualConstraints.update({key:({each}).union(individualConstraints[key])})
        elif re.match("[" + str(iMin) + "-" + str(iMax) + "]" + "[" + str(jMin) + "-" + str(jMax) + "]", key) and (test[int(key[0])][0] == 1 and test[int(key[1])][1] == 1) and  key != (str(unitSquare[0]) + str(unitSquare[1])):
            individualConstraints.update({key:({each}).union(individualConstraints[key])})

            
    rowConstraints[unitSquare[0]].append(each)
    columnConstraints[unitSquare[1]].append(each)
    squareConstraints[square].append(each)     
    
    #print("after")
    #print(individualConstraints)
    
    return individualConstraints
    
def solved(empty):
    if len(empty) == 0:
        return True
    else:
        return False
    
def finalInvalid(sudoku):
    if 0 in sudoku:
        return True
    else:
        return False

                                    
    
    

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

In [7]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 

This is your solution for medium sudoku number 9
[[3 6 2 4 9 5 8 1 7]
 [1 7 9 2 8 3 5 4 6]
 [4 5 8 6 1 7 9 3 2]
 [6 2 4 3 5 1 7 8 9]
 [5 8 1 9 7 2 4 6 3]
 [7 9 3 8 6 4 1 2 5]
 [9 3 5 1 2 8 6 7 4]
 [8 4 7 5 3 6 2 9 1]
 [2 1 6 7 4 9 3 5 8]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is medium sudoku number 10
[[3 4 6 7 2 5 9 8 1]
 [0 2 7 9 4 8 5 0 6]
 [9 5 0 0 1 6 7 0 2]
 [0 7 4 1 5 0 3 6 8]
 [8 0 0 0 3 7 1 5 4]
 [5 3 1 0 6 4 2 7 9]
 [6 9 0 4 7 1 8 0 5]
 [0 0 0 5 0 3 6 1 7]
 [0 0 5 6 8 0 4 9 3]]
This is your solution for medium sudoku number 10
[[3 4 6 7 2 5 9 8 1]
 [1 2 7 9 4 8 5 3 6]
 [9 5 8 3 1 6 7 4 2]
 [2 7 4 1 5 9 3 6 8]
 [8 6 9 2 3 7 1 5 4]
 [5 3 1 8 6 4 2 7 9]
 [6 9 3 4 7 1 8 2 5]
 [4 8 2 5 9 3 6 1 7]
 [7 1 5 6 8 2 4 9 3]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is medium sudoku number 11
[[6 8 0 1 7 5 3 4 9]
 [9 5 3 4 0 0 7 0 1]
 [0 1 0 2 3 0 5 6 0]
 [3 9 0 6 0 7 4 5 2]


This is your solution for hard sudoku number 9
[[5 9 4 2 8 7 1 3 6]
 [1 2 8 3 6 9 4 5 7]
 [7 6 3 5 1 4 8 2 9]
 [9 7 2 8 3 5 6 1 4]
 [4 8 6 7 2 1 5 9 3]
 [3 5 1 9 4 6 7 8 2]
 [8 3 5 6 7 2 9 4 1]
 [2 4 7 1 9 8 3 6 5]
 [6 1 9 4 5 3 2 7 8]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 1.890625 seconds to solve.

This is hard sudoku number 10
[[0 0 0 6 0 0 2 0 0]
 [0 0 0 0 0 9 0 6 0]
 [0 8 0 0 0 5 0 0 3]
 [1 0 0 4 0 0 9 0 0]
 [8 3 0 0 0 0 0 0 0]
 [0 2 0 0 0 6 0 0 0]
 [0 0 0 0 6 0 0 0 0]
 [2 5 0 3 0 7 0 9 0]
 [0 0 1 0 0 0 0 8 4]]
This is your solution for hard sudoku number 10
[[7 1 3 6 8 4 2 5 9]
 [5 4 2 7 3 9 8 6 1]
 [6 8 9 2 1 5 4 7 3]
 [1 7 6 4 5 3 9 2 8]
 [8 3 5 9 2 1 6 4 7]
 [9 2 4 8 7 6 3 1 5]
 [4 9 7 1 6 8 5 3 2]
 [2 5 8 3 4 7 1 9 6]
 [3 6 1 5 9 2 7 8 4]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 1.90625 seconds to solve.

This is hard sudoku number 11
[[0 0 0 5 2 0 3 0 0]
 [0 0 0 0 0 7 2 0 0]
 [6 0 8 0 3 0 0 0 1]
 [0 7 0 0 5 0 0 0 0]
 [1 

### Ignore Cell below (used for university submission)


In [4]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

You must set the SKIP_TESTS constant to True in the cell above.
You must include a separate file called readme.txt or readme.md in your submission.
This notebook file must be named sudoku.ipynb



Your submission is not ready! Please read and follow the instructions above.

In [ ]:
# This is a TEST CELL. Do not delete or change.